### Запуск QtLab

In [1]:
%run D:\qtlab_replacement\init.py
%matplotlib qt4
import matplotlib.pyplot as plt
from instruments import *
import numpy as np
import sweep
from save_pkl import *

In [2]:
#Параметры кубитов.
Qubits = {1: {'Fr': 9.5224e9,'F01': 6.5967e9, 'I0': 0.,'Ispan': 0.,'P': -40,'Pex': -20,'Span': 200e6}}

# Спектроскопия

In [3]:
pna = Agilent_N5242A('pna', address = 'PNA')
lo1 = Agilent_E8257D('lo1', address = 'PSG1')

In [5]:
# векторный анализатор в режим линейного свипа
pna.set_sweep_mode("LIN")

True

## Резонаторная спектроскопия

## Кубитная спектроскопия

In [23]:
#Кубит
q_id = 1

pna.set_average(0)
pna.set_averages(1)
pna.set_power(Qubits[q_id]['P'])
pna.set_nop(1)
pna.set_centerfreq(Qubits[q_id]['Fr'])
pna.set_bandwidth(5)

lo1.set_power(Qubits[q_id]['Pex'])
lo1.set_status(1)

True

### 1D скан при фиксированной мощности

In [26]:
#Измерение
ex_freqs = np.arange(6.3e9, 6.7e9, 1e6)
pna.set_power (-50)

header = '''#Parameter = Ex frequency
#Ex power = {:f}'''
try:
    ex1D = sweep.sweep(pna, (ex_freqs, lo1.set_frequency, 'Second tone freq'), filename = "ex_{:.1f}dBm".format(lo1.get_power()))
except KeyboardInterrupt: pass 

Started at:  Apr 25 2017 18:16:11
First sweep...

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Second tone freq: 6.699e+09,	average cycle time: 0.48s			
Elapsed time: 0 h 3 m 10.78 s


### 2D скан (мощность второго тона, частота второго тона)

In [8]:
ex_freqs = np.arange(5.5e9, 7e9, 10e6)
ex_powers = np.arange(-20, 15, 1)

pna.set_power(-50)
pna.set_bandwidth(10)
#lo1.set_power(-20)

header = '''#Parameter =
#Ro power = {:f}'''.format( pna.get_power() )

lo1.set_status(1)
#ps.S21vs2DPar(pna, currents, current.set_current, ex_freqs, lo1.set_frequency, filename = "#2{:.1f}dBm".format(lo1.get_power()),\
#           header = header)
ex2D = sweep.sweep(pna, (ex_powers, lo1.set_power, 'Second tone power'), (ex_freqs, lo1.set_frequency, 'Second tone freq'), filename = "#2{:.1f}dBm".format(pna.get_power()),\
           header = header)
lo1.set_status(0)

Started at:  Apr 25 2017 17:16:14


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Second tone power:     14,	Second tone freq: 6.99e+09,	average cycle time: 0.42s				
Elapsed time: 0 h 37 m 4.85 s


True

# Импульсы

### Создаём девайсы

In [3]:
if not ('pna' in locals()): pna = Agilent_N5242A('pna', address = 'PNA')
if not ('lo1' in locals()): lo1 = Agilent_E8257D('lo1', address = 'PSG1')
pxa = Agilent_N9030A('pxa', address = 'PXA')
awg_tek = Tektronix_AWG5014('awg_tek', address = 'TCPIP0::10.20.61.186::inst0::INSTR')
adc = Spectrum_M3i2132('adc')

root: WARNING  Tektronix_AWG5014 : get all not yet fully functional


### Загружаем импульсные скрипты всякие

In [146]:
import awg_digital
import awg_iq
import data_reduce
import imp
import fitting
import plotting
import save_pkl
import tomography
imp.reload(awg_digital)
imp.reload(awg_iq)
imp.reload(data_reduce)
imp.reload(fitting)
imp.reload(plotting)
imp.reload(save_pkl)
imp.reload(tomography)

<module 'tomography' from 'D:\\qtlab_replacement\\scripts\\tomography.py'>

### Осцилляторы

In [5]:
# Источник тока - в autorange
#current.set_autorange(1)
#Мощности гетеродинов, постоянные
#Мощность гетеродина для возбуждения 13-16 дБм
lo_ex_pow = 14
lo1.set_status(1)
lo1.set_power(lo_ex_pow)
#pna как lo2 (excitation)
#Port1 - выход
#Мощность гетеродина для считывания 10-13 дБм, с учётом разветвителя 13-16 dBm
lo_ro_pow = 16
pna.write("OUTP ON")
pna.write("SOUR1:POW1:MODE ON")
pna.write("SOUR1:POW2:MODE OFF")
pna.set_sweep_mode("CW")
pna.set_power(lo_ro_pow)

True

### AWGшки

In [6]:
# промежуточные частоты для гетеродинной схемы:
ex_if = 125e6
ro_if = 75e6
# клоки генератора и оцифровщика
ex_clock = 1e9
ro_clock = 1e9

rep_rate = 100e3 # частота повторений эксперимента

awg_tek.stop()
awg_tek.set_clock(ex_clock) # клок всех авгшк
awg_tek.set_nop(ex_clock/rep_rate) # репрейт нужно задавать по=хорошему только на управляющей,
awg_tek.run()
# а вот длину сэмплов, которая очевидно то же самое, нужно задавать на всех авгшках.
# хорошо, что сейчас она только одна.

# миксеры подключены к AWG5014 через 6 дБ аттенюаторы. Ставим амплитуду 2В.
for channel in range(1,5):
    awg_tek.set_amplitude(2, channel=channel)
# Выходы 1, 2 тектроникс: возбуждение (кубитный тон) через 6 дБ аттенюатор
# Выходы 3, 4 тектроникс: считывание (резонаторный тон) через 6 дБ аттенюатор
iq_ex = awg_iq.awg_iq(awg_tek, awg_tek, 1, 2, lo1)
iq_ro = awg_iq.awg_iq(awg_tek, awg_tek, 3, 4, pna)
iq_ex.set_if(ex_if)
iq_ro.set_if(ro_if)
iq_ex.set_sideband_id(-1)
iq_ro.set_sideband_id(-1)
iq_ex.set_frequency(Qubits[1]['F01'])
iq_ro.set_frequency(Qubits[1]['Fr'])

# К первому маркере первого канала тектроникса подсоединён триггер оцифровщика.
# Надо подавать какой-нибудь триггер, что-ли.
# А ещё нужно подавать клок. К сожалению наш оцифровщик не может в 1ГГц клок, ему нужен 500МГц клок.
# Будем подавать 500 МГц с цифрового выхода тектроникса.
awg_tek.set_marker1_low(-1, channel=1)
awg_tek.set_marker2_low(-1, channel=1)
awg_tek.set_marker1_high(2, channel=1)
awg_tek.set_marker2_high(2, channel=1)
awg_tek.set_marker1_low(-0.1, channel=2)
awg_tek.set_marker2_low(-0.1, channel=2) 
awg_tek.set_marker1_high(0.1, channel=2) # с клока хватит и такой амплитуды
awg_tek.set_marker2_high(0.1, channel=2) # с клока хватит и такой амплитуды
ro_trg = awg_digital.awg_digital(awg_tek, 1)
osc_trg = awg_digital.awg_digital(awg_tek, 5)
ro_adc_clock = awg_digital.awg_digital(awg_tek, 2) #
osc_adc_clock = awg_digital.awg_digital(awg_tek, 6) #

# тупо клок
ro_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
osc_adc_clock.set_waveform(np.asarray([0,1]*int(ro_adc_clock.get_nop()/2), dtype=int))
awg_channels = {'iq_ex':iq_ex, 'iq_ro':iq_ro, 'ro_trg':ro_trg, 'osc_trg':osc_trg}

root: WARNING  Tektronix_AWG5014 : Clock set to 1000000000.0. This is not fully functional yet. To avoid problems, it is better not to change the clock during operation


### Оцифровщик

In [7]:
# настройки оцифровщика
adc.set_clock(ro_clock)
adc.reset()            
adc.set_timeout(10000)


#adc.set_spc_samplerate(smp_rate)
#External trigger
adc.set_trigger_ext0_level0(0) # logical 0 value to 200 mV
adc.set_trigger_ext0_level1(1300) # logical 1 value to 600 mV
adc.trigger_termination_50Ohm()
adc.trigger_mode_pos()
adc.set_trigger_ext0_pulsewidth(0)
adc.disable_trigger_output()
adc.select_channel01()
adc.set_multi_mode()
#adc.set_segmentsize()
adc.set_trigger_ORmask_tmask_ext0()
adc.set_trigger_ANDmask_tmask_ext0()
adc.set_trigger_ORmask_tmask_NO_ch0()
adc.set_trigger_ORmask_tmask_NO_ch1()
adc.set_trigger_ANDmask_tmask_NO_ch0()
adc.set_trigger_ANDmask_tmask_NO_ch1()
# усилитель на оцифровщике
adc.set_input_amp_ch0(50)
adc.set_input_amp_ch1(50)
adc.set_input_offset_ch0(0)
adc.set_input_offset_ch1(0)

#External clock
adc.set_reference_clock(adc.get_clock())

# "Измеритель средних прошедших импульсов"
# Просто усредняет всё по номеру сэмпла.
adc_reducer = data_reduce.data_reduce(adc)
adc_reducer.filters['Mean Voltage (AC)'] = data_reduce.mean_reducer_noavg(adc, 'Voltage', 0)
adc_reducer.filters['S21+'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, iq_ro.get_if())
adc_reducer.filters['S21-'] = data_reduce.mean_reducer_freq(adc, 'Voltage', 0, -iq_ro.get_if())

### Калибровка миксеров

In [8]:
iq_ex.get_calibration(pxa)
iq_ro.get_calibration(pxa)

d:\qtlab\ReiData\data/calibrations//IQ-if1.2e+08-rf6.6e+09-sb--1.pkl
d:\qtlab\ReiData\data/calibrations//IQ-if7.5e+07-rf9.5e+09-sb--1.pkl


{'I': (0.30274495983806538+0.47960482672767657j),
 'Q': (0.59928899400797109+0.57211194162260182j),
 'dc': (0.013841661224179503-0.024369994098197508j),
 'num_sidebands': 7,
 'score': -4.0290860342648021}

### Функции для создания последовательностей импульсов и кирпичи

In [9]:
def set_adc_nop(ro_adc_length):
    adc.stop()
    signal_size = int(np.ceil(4./3.*(ro_adc_length)*adc.get_clock()))
    nop = int( 2**np.ceil(np.log2(signal_size)) )
    posttrigger = nop*3/4
    
    print (nop)
    adc.set_nop(nop)
    adc.set_post_trigger(posttrigger)

### Длительность и амплитуда считывающих импульсов

In [60]:
ro_adc_length = 0.6e-6        # Вот такой длины считываем (опухоль Фёдорова!!)
ro_amplitude = 0.5
ro_dac_length = 0.4e-6
adc.set_nums(5000)
adc.set_software_nums_multi(4)
set_adc_nop(ro_adc_length)

512


In [11]:
plt.plot(np.mean(np.real(adc.measure()['Voltage']), axis=0))

In [12]:
adc.stop()

### Прохождение считывающих испульсов разной амплитуды

In [12]:
def set_pulse_amplitude(x):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    tomography.set_sequence([tomography.ro_rect(x, ro_dac_length, awg_channels)], awg_channels)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (np.linspace(0, 1, 51), set_pulse_amplitude, 'Readout amplitude'), filename='Readout pulse passthrough')

Started at:  Apr 27 2017 14:39:06


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 1 m 14.68 s,	parameter values: Readout amplitude:   0.38,	average cycle time:  2.4s	

KeyboardInterrupt: 

KeyboardInterrupt: 

### Осцилляции Раби

In [14]:
ex_amplitude = 0.25
pause_length = 16e-9
lengths = np.linspace(0e-9, 100e-9, 101)
readout_begin = np.max(lengths)
def set_ex_length(length):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [tomography.pause(readout_begin-length, awg_channels), 
                tomography.ex_rect(ex_amplitude, length, awg_channels), 
                tomography.pause(pause_length, awg_channels), 
                tomography.ro_rect(ro_amplitude, ro_dac_length, awg_channels)]
    tomography.set_sequence(sequence, awg_channels)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (lengths, set_ex_length, 'Rabi pulse length'), filename='Rabi')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl({'type':'Rabi'}, measurement_fitted)

Started at:  Apr 26 2017 15:48:05


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Rabi pulse length:  1e-07,	average cycle time:  2.2s	s		
Elapsed time: 0 h 3 m 44.12 s


### Осцилляции Рамзея

In [17]:
ex_amplitude = 0.23
pi2_pulse = 12e-9
pause_length = 16e-9
delays = np.linspace(0e-9, 200e-9, 41)
target_freq_offset = 20e6

readout_begin = np.max(delays)+pi2_pulse*2

def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [tomography.pause(readout_begin-delay, awg_channels),
                tomography.ex_rect(ex_amplitude, pi2_pulse, awg_channels),
                tomography.pause(delay, awg_channels),
                tomography.ex_rect(ex_amplitude*np.exp(1j*delay*target_freq_offset*2*np.pi), pi2_pulse, awg_channels), 
                tomography.pause(pause_length, awg_channels), 
                tomography.ro_rect(ro_amplitude, ro_dac_length, awg_channels)]
    tomography.set_sequence(sequence, awg_channels)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl({'type':'Ramsey'}, measurement_fitted)

Started at:  Apr 26 2017 15:54:07


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 8.91 s,	parameter values: Ramsey delay: 1.8e-07,	average cycle time:  2.2s	s	

KeyboardInterrupt: 

### Затухание свободной индукции (T1)

In [ ]:
ex_amplitude = 0.25
pi_pulse = 46e-9
pause_length = 16e-9
delays = np.linspace(0e-9, 4000e-9, 41)
readout_begin = np.max(delays)+pi_pulse
def set_delay(delay):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [tomography.pause(readout_begin-delay, awg_channels),
                tomography.ex_rect(ex_amplitude,pi_pulse, awg_channels),
                tomography.pause(delay, awg_channels),
                tomography.ro_rect(ro_amplitude, ro_dac_length, awg_channels)]
    tomography.set_sequence(sequence, awg_channels)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'delay'), filename='Decay')

measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_fit)
save_pkl({'type':'Decay'}, measurement_fitted)
print ('Decay: {0:4.6g} s'.format(fitted_parameters['decay']))

### Rabi amplitude oscillations

In [55]:
ex_amps = np.linspace(0.,0.5,51)
pause_length = 32e-9
length = 96e-9
sigma = 24e-9
num_pulses = 4
readout_begin = np.max((length+pause_length)*num_pulses)
def set_ex_amp(amp):
    awg_tek.set_nop(awg_tek.get_clock()/rep_rate)
    sequence = [tomography.ex_gauss(amp, length, sigma, awg_channels), 
                tomography.pause(pause_length, awg_channels)]*num_pulses+[tomography.ro_rect(ro_amplitude, ro_dac_length, awg_channels)]
    tomography.set_sequence(sequence, awg_channels)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (ex_amps, set_ex_amp, 'Rabi pulse amplitude'), filename='Rabi')

Started at:  Apr 27 2017 16:20:21


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 1 m 13.36 s,	parameter values: Rabi pulse amplitude:   0.31,	average cycle time:  3.9s	

KeyboardInterrupt: 

### gaussian $\pi/2$ pulse amplitudes

In [13]:
#amp_x=0.554
#amp_y=0.554
#sigma = 2e-9
#amp_x=0.29
#amp_y=0.29
amp_x=0.600/4
amp_y=0.600/4*np.exp(1j*1.65)
length = 96e-9
pause_length = 32e-9
sigma = 24e-9
amp_x_hd = 0.6
amp_y_hd = 0.6*np.exp(1j*1.65)
length_hd = 24e-9
sigma_hd = 6e-9
pause_length_hd = 8e-9

### Ramsey with gauss pulses

In [12]:
delays = np.linspace(0e-9, 2000e-9, 201)
target_freq_offset = 20e6

def set_delay(delay):
    sequence = [tomography.ex_gauss(amp_x, length, sigma, awg_channels),
                tomography.pause(delay, awg_channels),
                tomography.ex_gauss(amp_x*np.exp(1j*delay*target_freq_offset*2*np.pi), length, sigma, awg_channels),
                tomography.pause(pause_length, awg_channels), 
                tomography.ro_rect(ro_amplitude, ro_dac_length, awg_channels)]
    tomography.set_sequence(sequence, awg_channels)
    awg_tek.run()
measurement = sweep.sweep(adc_reducer, (delays, set_delay, 'Ramsey delay'), filename='Ramsey')
measurement_fitted, fitted_parameters = fitting.S21pm_fit(measurement, fitting.exp_sin_fit)
annotation = 'Phase: {0:4.4g} rad, Freq: {1:4.4g}, Decay: {2:4.4g} s'.format(fitted_parameters['phase'], 
                                                                             fitted_parameters['freq'], 
                                                                             fitted_parameters['decay'])
save_pkl.save_pkl({'type':'Ramsey'}, measurement_fitted, annotation=annotation)

Started at:  Apr 28 2017 11:54:13


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 8 m 26.36 s,	parameter values: Ramsey delay:      0,	average cycle time:  2.5s	

KeyboardInterrupt: 

### |0> & |1> readout calibration

In [61]:
ex_calib_seq = [tomography.ex_gauss(amp_x, length, sigma, awg_channels), tomography.pause(pause_length, awg_channels)]*2
ro_seq = [tomography.ro_rect(ro_amplitude, ro_dac_length, awg_channels)]
adc_sz = tomography.sz_measurer(adc, ex_calib_seq, ro_seq, awg_channels)
adc.set_software_nums_multi(20)
sz_filter = adc_sz.calibrate()
adc.set_software_nums_multi(1)
adc_sz.calib_roc_auc_score

D:\qtlab_replacement\scripts\tomography.py:134: RuntimeWarning: invalid value encountered in true_divide
  probabilities = hists/hist_all


0.71742644289999991

In [63]:
#plt.scatter(np.real(adc_sz.calib_usl_pred[0,:]), np.imag(adc_sz.calib_usl_pred[0,:]), color='r', s=1)
#plt.scatter(np.real(adc_sz.calib_usl_pred[1,:]), np.imag(adc_sz.calib_usl_pred[1,:]), color='b', s=1)
plt.figure()
plt.hist(adc_sz.calib_pred.T)
plt.figure()
plt.hist(np.real(adc_sz.calib_usl_pred.T))

([array([  3.60000000e+01,   7.42000000e+02,   6.23600000e+03,
           2.33440000e+04,   3.58200000e+04,   2.45840000e+04,
           7.85600000e+03,   1.24500000e+03,   1.31000000e+02,
           6.00000000e+00]),
  array([  1.10000000e+01,   3.15000000e+02,   3.27900000e+03,
           1.45590000e+04,   3.05670000e+04,   3.11530000e+04,
           1.56110000e+04,   3.98900000e+03,   4.92000000e+02,
           2.40000000e+01])],
 array([-12.33122934, -10.24868929,  -8.16614924,  -6.08360919,
         -4.00106914,  -1.91852909,   0.16401096,   2.24655101,
          4.32909107,   6.41163112,   8.49417117]),
 <a list of 2 Lists of Patches objects>)

In [138]:
plt.plot(adc_sz.calib_usl_cov[:,-3:])

c:\python36\lib\site-packages\numpy\core\numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [21]:
plt.pcolormesh(np.log10(np.abs(adc_sz.calib_usl_cov)))

In [117]:
W,V = np.linalg.eigh(adc_sz.calib_usl_cov)

In [132]:
plt.pcolormesh(np.log10(np.abs(V)), vmin=-5, vmax=-1)

c:\python36\lib\site-packages\ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log10
  """Entry point for launching an IPython kernel.


In [141]:
plt.semilogy(np.abs(np.dot(adc_sz.calib_usl_cov,V[:, 512])))
plt.semilogy(np.abs(V[:, 512]))
plt.semilogy(np.abs(np.dot(adc_sz.calib_usl_cov,V[:, 511])))
plt.semilogy(np.abs(V[:, 511]))

In [45]:
mean_ind = np.argsort(np.abs(W))[-1]
diff_ind = np.argsort(np.abs(W))[-2]
plt.plot(V[mean_ind,:])
plt.plot(V[diff_ind,:])

c:\python36\lib\site-packages\numpy\core\numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [64]:
plt.figure()
plt.plot(np.real(adc_sz.calib_bg-np.mean(adc_sz.calib_bg)), label='BG from SL')
plt.plot(np.real(adc_sz.calib_usl_bg), label='BG from USL')
plt.plot(np.real(adc_sz.calib_feature), label='feature from SL')
plt.plot(np.real(adc_sz.calib_usl_feature), label='feature from USL')
plt.legend()

In [52]:
np.dot(adc_sz.calib_usl_feature, adc_sz.calib_feature)/(np.sum(np.abs(adc_sz.calib_usl_feature)**2)*np.sum(np.abs(adc_sz.calib_feature)**2))**0.5

(0.42692336271427372+0.071258130967687044j)

In [46]:
plt.plot(np.abs(np.fft.fft(adc_sz.calib_feature))**2)

In [53]:
plt.plot(adc_sz.calib_proba_points, adc_sz.calib_hists.T/np.max(adc_sz.calib_hists))
plt.plot(adc_sz.calib_proba_points, adc_sz.calib_proba)

### Tomography

In [42]:
proj_seq = {'X':[tomography.ex_gauss(-amp_y, length, sigma, awg_channels), tomography.pause(pause_length, awg_channels)]+ro_seq,
            'Y':[tomography.ex_gauss(amp_x, length, sigma, awg_channels), tomography.pause(pause_length, awg_channels)]+ro_seq,
            '-X':[tomography.ex_gauss(amp_y, length, sigma, awg_channels), tomography.pause(pause_length, awg_channels)]+ro_seq,
            '-Y':[tomography.ex_gauss(-amp_x, length, sigma, awg_channels), tomography.pause(pause_length, awg_channels)]+ro_seq,
            'Z':ro_seq}
tomo = tomography.tomography(adc, adc_sz, awg_channels, proj_seq)

In [65]:
pause_and_ro = [tomography.pause(pause_length, awg_channels)]+ro_seq
proj_seq = {'X':[tomography.ex_gauss_hd(-amp_y_hd, amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi)]+pause_and_ro,
            'Y':[tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi)]+pause_and_ro,
            '-X':[tomography.ex_gauss_hd(amp_y_hd, -amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi)]+pause_and_ro,
            '-Y':[tomography.ex_gauss_hd(-amp_x_hd, -amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi)]+pause_and_ro,
            'Z':ro_seq}
tomo_hd = tomography.tomography(adc, adc_sz, awg_channels, proj_seq)

In [61]:
proj_seq = {'Z':ro_seq}
tomoz = tomography.tomography(adc, adc_sz, awg_channels, proj_seq)

In [73]:
## tomography of |0> state
adc.set_software_nums_multi(8)
tomo_hd.set_prepare_seq([])
print(tomo_hd.measure())
tomo.set_prepare_seq([])
print(tomo.measure())

{'X': array(0.05512057436072596), 'Y': array(0.05486218480068086), '-X': array(0.0621648864182589), '-Y': array(0.02619638164967206), 'Z': array(-0.47885208959108444)}
{'X': array(0.0724993164386453), 'Y': array(0.05151829014084406), '-X': array(0.07318903004467359), '-Y': array(0.08122722030067299), 'Z': array(-0.44259693331581723)}


In [44]:
## tomography of |1> state
tomo.set_prepare_seq([tomography.ex_gauss(amp_x, length, sigma, awg_channels), tomography.pause(pause_length, awg_channels)]*2)
tomo.measure()

{'-X': array(0.041529804226347326),
 '-Y': array(0.06224032031912466),
 'X': array(0.09107830970740463),
 'Y': array(0.09390934842531314),
 'Z': array(0.5033204860342512)}

In [45]:
## tomography of |0>+|1> state
tomo.set_prepare_seq([tomography.ex_gauss(amp_x, length, sigma, awg_channels), tomography.pause(pause_length, awg_channels)])
tomo.measure()

{'-X': array(-0.006969356543138275),
 '-Y': array(-0.4705261020624773),
 'X': array(0.14556522294921176),
 'Y': array(0.5152869647143166),
 'Z': array(0.06329995599464514)}

In [46]:
## tomography of |0>-|1> state
tomo.set_prepare_seq([tomography.ex_gauss(1j*amp_y, length, sigma, awg_channels), tomography.pause(pause_length, awg_channels)])
tomo.measure()

{'-X': array(0.1064681809587594),
 '-Y': array(0.4988969810695382),
 'X': array(0.03969676914166282),
 'Y': array(-0.46678261858203707),
 'Z': array(0.0550630861513214)}

In [47]:
## tomography of -|0>-|1> state
tomo.set_prepare_seq([tomography.ex_gauss(-amp_x, length, sigma, awg_channels), tomography.pause(pause_length, awg_channels)])
tomo.measure()

{'-X': array(0.12654330146641085),
 '-Y': array(0.5281811933884821),
 'X': array(-0.01845174336429209),
 'Y': array(-0.451163415229694),
 'Z': array(0.06393654590519757)}

In [48]:
## tomography of -|0>+|1> state
tomo.set_prepare_seq([tomography.ex_gauss(-1j*amp_y, length, sigma, awg_channels), tomography.pause(pause_length, awg_channels)])
tomo.measure()

{'-X': array(0.03474817792333144),
 '-Y': array(-0.4610157800017896),
 'X': array(0.10479361837368074),
 'Y': array(0.4563621744875563),
 'Z': array(0.048932335531325706)}

### Randomized Benchmarking

In [157]:
pause_pulse = tomography.pause(pause_length, awg_channels)
pulses = {0:tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          1:tomography.ex_gauss_hd(amp_y_hd, -amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          2:tomography.ex_gauss_hd(-amp_x_hd, -amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi),
          3:tomography.ex_gauss_hd(-amp_y_hd, amp_x_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi)}

unitaries = {0:np.sqrt(0.5)*np.asarray([[1, -1j],  [-1j, 1]]),\
             1:np.sqrt(0.5)*np.asarray([[1, -1],    [1, 1]]),\
             2:np.sqrt(0.5)*np.asarray([[1, 1j],   [1j, 1]]),\
             3:np.sqrt(0.5)*np.asarray([[1, 1],   [-1, 1]])}

observables = { 'X': 0.5*np.asarray([[0, 1],   [1, 0]]),
                'Y': 0.5*np.asarray([[0, -1j],   [1j, 0]]),
                '-X': 0.5*np.asarray([[0, -1],   [-1, 0]]),
                '-Y': 0.5*np.asarray([[0, 1j],   [-1j, 0]]),
                'Z': 0.5*np.asarray([[1, 0],   [0, -1]])}

initial = np.asarray([0, 1]).T
random_pulse_ids = np.arange(20)
num_gates = np.arange(21)

sequences = []
pulse_sequences = []
theor_projs = {o:np.zeros((len(num_gates), len(random_pulse_ids))) for o in observables.keys()}
rho_t = np.zeros((len(num_gates), len(random_pulse_ids),2,2), dtype=np.complex)
psis = np.zeros((len(num_gates), len(random_pulse_ids),2), dtype=np.complex)
n=0

def set_num_gates(n_new):
    global n
    n = n_new

def set_random_sequence(id):
    sequence = np.random.randint (4, size = n).tolist()
    sequence_pulses = [j for i in [[pulses[i], pause_pulse] for i in sequence] for j in i]
    sequence_unitaries = [unitaries[i] for i in sequence]
    psi = initial.copy()
    
    for U in sequence_unitaries:
        psi = np.dot(U, psi)
    
    rho_t[n,id,:,:] = np.einsum('i,j->ij', np.conj(psi), psi)
    psis[n,id,:] = psi
    
    sequences.append(sequence)
    for o in observables.keys():
        theor_projs[o][n,id] = np.dot(np.dot(np.conj(psi), observables[o]), psi)
    
    tomo_hd.set_prepare_seq(sequence_pulses)
    pulse_sequences.append(sequence_pulses)
    
m = sweep.sweep(tomo_hd, (num_gates, set_num_gates, 'gate num'), (random_pulse_ids, set_random_sequence, 'random seq id'), filename='RB', output=False)
X = (m['X'][2]-m['-X'][2])/2.
Y = (m['Y'][2]-m['-Y'][2])/2.
Z = m['Z'][2]
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('nmi,nmji,nmj->nm', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

Started at:  Apr 28 2017 16:50:15


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


First sweep...

c:\python36\lib\site-packages\ipykernel_launcher.py:44: ComplexWarning: Casting complex values to real discards the imaginary part


Time left: 0 h 0 m 0.0 s,	parameter values: gate num:     20,	random seq id:     19,	average cycle time:   14s				
Elapsed time: 1 h 41 m 29.59 s


c:\python36\lib\site-packages\numpy\core\numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [ ]:
plt.plot(m['Fidelity'][1]m['Fidelity'])

In [154]:
rho_m

array([[[[ -4.54105167e-03+0.j        ,  -6.70718763e-03-0.01268007j],
         [ -6.70718763e-03+0.01268007j,   1.00454105e+00+0.j        ]],

        [[  1.90080873e-02+0.j        ,  -7.31600466e-04-0.00128394j],
         [ -7.31600466e-04+0.00128394j,   9.80991913e-01+0.j        ]]],


       [[[  5.23648939e-01+0.j        ,   1.67812796e-02+0.45229873j],
         [  1.67812796e-02-0.45229873j,   4.76351061e-01+0.j        ]],

        [[  5.34648328e-01+0.j        ,   9.83864179e-03+0.46630912j],
         [  9.83864179e-03-0.46630912j,   4.65351672e-01+0.j        ]]]])

In [168]:
plt.plot(m['Fidelity'][1][0], np.mean(m['Fidelity'][2],axis=1))
plt.xlabel('gate num')
plt.ylabel('fidelity')
plt.grid()

In [152]:
rho_m = np.transpose([[0.5+Z, X+1j*Y], [X-1j*Y, 0.5-Z]], [2,3,0,1])
fidelities = np.sqrt(np.einsum('mni,mnji,mnj->mn', np.conj(psis), rho_m, psis))

m['Fidelity'] = [d for d in m['X']]
m['Fidelity'][2] = np.asarray(fidelities, dtype=np.float)
for o,O in observables.items():
    m[o+' fit'] = [d for d in m[o]]
    m[o+' fit'][2] = np.asarray(theor_projs[o], dtype=np.float)
#m['rho_t'] = m['X'].copy()
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, m)

c:\python36\lib\site-packages\numpy\core\numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


In [143]:
m

{'-X': (('random seq id', 'gate num'),
  (array([0, 1]), array([0, 1])),
  array([[ 0.02071368,  0.06280285],
         [-0.47004341, -0.47692684]]),
  {}),
 '-X fit': [('random seq id', 'gate num'),
  (array([0, 1]), array([0, 1])),
  array([[ 0. ,  0. ],
         [ 0. , -0.5]]),
  {}],
 '-Y': (('random seq id', 'gate num'),
  (array([0, 1]), array([0, 1])),
  array([[ 0.02339924,  0.07005866],
         [ 0.01224385,  0.00593551]]),
  {}),
 '-Y fit': [('random seq id', 'gate num'),
  (array([0, 1]), array([0, 1])),
  array([[ 0.,  0.],
         [ 0.,  0.]]),
  {}],
 'Fidelity': [('random seq id', 'gate num'),
  (array([0, 1]), array([0, 1])),
  array([[ 0.99258696,  0.        ],
         [ 0.        ,  0.99811094]]),
  {}],
 'X': (('random seq id', 'gate num'),
  (array([0, 1]), array([0, 1])),
  array([[ 0.03073913,  0.03102173],
         [ 0.50264322,  0.51552406]]),
  {}),
 'X fit': [('random seq id', 'gate num'),
  (array([0, 1]), array([0, 1])),
  array([[ 0. ,  0. ],
         [ 0

In [122]:
fidelities

array([ 0.97991445+0.j,  0.93312074+0.j,  0.97351467+0.j,  0.88195572+0.j,
        0.98395691+0.j])

In [123]:
psis

[array([-0.5+0.5j, -0.5-0.5j]),
 array([ 0.5-0.5j, -0.5-0.5j]),
 array([ 0.5+0.5j,  0.5+0.5j]),
 array([ 0.5-0.5j, -0.5+0.5j]),
 array([-0.5+0.5j, -0.5+0.5j])]

In [124]:
rho_m

array([[[ 0.59661779+0.j        ,  0.11373132+0.46023232j],
        [ 0.11373132-0.46023232j,  0.40338221+0.j        ]],

       [[ 0.54508677+0.j        ,  0.06986417-0.37071432j],
        [ 0.06986417+0.37071432j,  0.45491323+0.j        ]],

       [[ 0.68233096+0.j        ,  0.44773082+0.08804433j],
        [ 0.44773082-0.08804433j,  0.31766904+0.j        ]],

       [[ 0.45918886+0.j        , -0.27784590+0.18198637j],
        [-0.27784590-0.18198637j,  0.54081114+0.j        ]],

       [[ 0.56393048+0.j        ,  0.46817119+0.00919325j],
        [ 0.46817119-0.00919325j,  0.43606952+0.j        ]]])

In [125]:
rho_t

array([[[ 0.5+0.j ,  0.0+0.5j],
        [ 0.0-0.5j,  0.5+0.j ]],

       [[ 0.5+0.j ,  0.0-0.5j],
        [ 0.0+0.5j,  0.5+0.j ]],

       [[ 0.5+0.j ,  0.5+0.j ],
        [ 0.5+0.j ,  0.5+0.j ]],

       [[ 0.5+0.j , -0.5+0.j ],
        [-0.5+0.j ,  0.5+0.j ]],

       [[ 0.5+0.j ,  0.5+0.j ],
        [ 0.5+0.j ,  0.5+0.j ]]])

In [198]:
measurement['theory'] = thoer_proj
save_pkl.save_pkl({'type':'Randomized benchmark Clifford'}, measurement)

KeyError: 'Sequences'

In [201]:
print(theor_projs)
print(measurement)
print(psis)
print (sequences)
print (rho_t)
print (rho_m)

[{'X': 0j, 'Y': 0j, '-X': 0j, '-Y': 0j, 'Z': (0.50000000000000022+0j)}, {'X': 0j, 'Y': 0j, '-X': 0j, '-Y': 0j, 'Z': (0.50000000000000022+0j)}, {'X': (0.50000000000000022+0j), 'Y': 0j, '-X': (-0.50000000000000022+0j), '-Y': 0j, 'Z': 0j}, {'X': 0j, 'Y': (-0.50000000000000022+0j), '-X': 0j, '-Y': (0.50000000000000022+0j), 'Z': 0j}, {'X': 0j, 'Y': (-0.50000000000000022+0j), '-X': 0j, '-Y': (0.50000000000000022+0j), 'Z': 0j}, {'X': (-0.50000000000000022+0j), 'Y': 0j, '-X': (0.50000000000000022+0j), '-Y': 0j, 'Z': 0j}, {'X': 0j, 'Y': (-0.50000000000000022+0j), '-X': 0j, '-Y': (0.50000000000000022+0j), 'Z': 0j}, {'X': 0j, 'Y': 0j, '-X': 0j, '-Y': 0j, 'Z': (-0.50000000000000022+0j)}, {'X': (0.50000000000000022+0j), 'Y': 0j, '-X': (-0.50000000000000022+0j), '-Y': 0j, 'Z': 0j}, {'X': (0.50000000000000022+0j), 'Y': 0j, '-X': (-0.50000000000000022+0j), '-Y': 0j, 'Z': 0j}]
{'X': (('random seq id',), (array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),), array([ 0.13529069,  0.03693103,  0.46135948,  0.09578221,

In [194]:
np.dot(np.conj(psis[1]), np.dot(observables['Y'],psis[1]))

(-0.50000000000000022+0j)

In [71]:
num_pulses = np.round(np.arange(0, 41)).astype(np.int)

def set_num_pulses(n):
    tomoz.set_prepare_seq([tomography.ex_gauss(amp_x, length, sigma, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomoz, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

Started at:  Apr 27 2017 16:48:18
First sweep...

c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: SY pi2 pulse num:     40,	average cycle time:  3.5s			
Elapsed time: 0 h 2 m 24.22 s


In [115]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, -200e6/np.pi), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

Started at:  Apr 27 2017 20:15:31


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: SX pi2 pulse num:     20,	average cycle time:   18s	s	
Elapsed time: 0 h 6 m 16.75 s


In [110]:
num_pulses = np.round(np.arange(0, 21)).astype(np.int)

def set_num_pulses(n):
    tomo.set_prepare_seq([tomography.ex_gauss(amp_x_hd, length_hd, sigma_hd, awg_channels), 
                          tomography.pause(pause_length, awg_channels)]*n)

measurement = sweep.sweep(tomo, (num_pulses, set_num_pulses, 'SX pi2 pulse num'), filename='Rabi', output=False)

Started at:  Apr 27 2017 19:39:12


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: SX pi2 pulse num:     20,	average cycle time:   18s	s	
Elapsed time: 0 h 6 m 14.09 s


In [108]:
num_pulses = 2
deltas = np.arange(-350,-150,10)*1e6
def set_anharm_freq (f):
     tomo.set_prepare_seq([tomography.ex_gauss_hd(amp_x_hd, amp_y_hd, length_hd, sigma_hd, awg_channels, f), 
                          tomography.pause(pause_length, awg_channels)]*num_pulses)
measurement = sweep.sweep(tomo, (deltas,set_anharm_freq, 'delta_freq'), filename='DRAG', output=False)

Started at:  Apr 27 2017 19:24:26


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: delta_freq: -1.6e+08,	average cycle time:   18s	s	
Elapsed time: 0 h 5 m 55.94 s


In [79]:
ex_amplitude=0.25
lengths = np.linspace(0e-9, 200e-9, 51)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi x-axis pulse length'), filename='Rabi', output=False)

Started at:  Apr 27 2017 17:39:03


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Rabi x-axis pulse length:  3e-07,	average cycle time:   18s	18s	
Elapsed time: 0 h 15 m 21.47 s


In [85]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot(measurement ['X'][2], measurement ['Y'][2], measurement ['Z'][2], c='r', marker='o')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim([-0.6, 0.6])
ax.set_ylim([-0.6, 0.6])
ax.set_zlim([-0.6, 0.6])

(-0.6, 0.6)

In [57]:
ex_amplitude = 0.25
pause_length = 16e-9
lengths = np.linspace(0e-9, 2000e-9, 2001)
readout_begin = np.max(lengths)

def set_ex_length(length):
    tomo.set_prepare_seq([tomography.ex_rect(1j*ex_amplitude, length, awg_channels), tomography.pause(pause_length, awg_channels)])

measurement = sweep.sweep(tomo, (lengths, set_ex_length, 'Rabi y-axis pulse length'), filename='Rabi', output=False)

Started at:  Apr 27 2017 00:17:26


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 0 m 0.0 s,	parameter values: Rabi y-axis pulse length:  2e-06,	average cycle time:   10s	 10s		
Elapsed time: 5 h 33 m 44.24 s


### Optimizing phase between x and y pulses

In [111]:
c_phase = 0
c_n = 0

adc.set_software_averages(1)
adc.set_software_nums_multi(3)

def set_n(n):
    global c_n
    global c_phase
    c_n = n 
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

def set_y_phase(phase):
    global c_n
    global c_phase
    c_phase = phase
    y_pulse = [tomography.ex_gauss_hd(amp_x_hd*np.exp(1j*c_phase), amp_y_hd*np.exp(1j*c_phase), length_hd, sigma_hd, awg_channels, -200e6), tomography.pause(pause_length, awg_channels)]
    tomo.set_prepare_seq(y_pulse*c_n)

phases = np.linspace(np.pi/2.*0.8, np.pi/2.*1.2, 11, endpoint=False)
num_pulses = np.arange(0,5)
measurement = sweep.sweep(tomo, (phases, set_y_phase, 'y-pulse phase'), (num_pulses, set_n, 'y pulse num'), filename='y-phase', output=False)

Started at:  Apr 27 2017 19:45:43


c:\python36\lib\site-packages\matplotlib\backend_bases.py:2445: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


Time left: 0 h 3 m 54.74 s,	parameter values: y-pulse phase:  1.714,	y pulse num:      1,	average cycle time:   18s		

KeyboardInterrupt: 

In [73]:
phases[7]

1.6564761264382546

In [117]:
plt.plot(np.asarray(results).T)

c:\python36\lib\site-packages\numpy\core\numeric.py:531: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


In [139]:
adc.stop()

In [88]:
plt.close()